<a href="https://colab.research.google.com/github/barikosan/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab Setup

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install all the necessary packages (Change it with your working directory )
!sh /content/drive/MyDrive/RAG/install_packages.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.8/643.8 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.2/531.2 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving spark_nlp_for_healthcare_spark_ocr_8514.json to spark_nlp_for_healthcare_spark_ocr_8514.json


In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8514.json
🚨 Outdated Medical Secrets in license file. Version=5.1.3 but should be Version=5.1.0
🚨 Outdated OCR Secrets in license file. Version=5.0.2 but should be Version=5.0.1
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8514.json
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-5.1.0-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==5.1.0 installed! ✅ Heal the planet with NLP! 


In [ ]:
# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8514.json
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.1.0, 💊Spark-Healthcare==5.1.0, running on ⚡ PySpark==3.1.2


In [ ]:
spark

In [ ]:
# required libraries
import pandas as pd
import ast

from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

## Step 1: Retrieval

### Data collection

In [ ]:
# imports
import wikipediaapi

In [ ]:
def get_wiki_data(page_title):
  content = ""
  # Initialize Wikipedia API
  wiki_wiki = wikipediaapi.Wikipedia('MentalHealthProject','en')  # 'en' for English language
  # Load the page
  page = wiki_wiki.page(page_title)
  # Check if the page exists
  if page.exists():
    # Print the content
    content = page.text
  else:
    print(f"The page '{page_title}' does not exist.")

  return content

In [ ]:
data = get_wiki_data("Mental health")
print(data)

Mental health encompasses emotional, psychological, and social well-being, influencing cognition, perception, and behavior. According to World Health Organization (WHO), it is a "state of well-being in which the individual realizes his or her abilities, can cope with the normal stresses of life, can work productively and fruitfully, and can contribute to his or her community". It likewise determines how an individual handles stress, interpersonal relationships, and decision-making. Mental health includes subjective well-being, perceived self-efficacy, autonomy, competence, intergenerational dependence,  and self-actualization of one's intellectual and emotional potential, among others. From the perspectives of positive psychology or holism, mental health may include an individual's ability to enjoy life and to create a balance between life activities and efforts to achieve psychological resilience. Cultural differences, subjective assessments, and competing professional theories all af

In [ ]:
# manually curated data for questions relevant to that page
#q_data = 'What is the definition of mental health? Who was the first person to coin the term mental hygiene? What percentage of Americans suffer from a mental illness? What is the role of stress in mental health problems? How pharmacotherapy is used in the treatment of mental diseases? '
q_data = '''How World Health Organization(WHO) defines mental health?
            Who was the first person to coin the term mental hygiene?
            What percentage of Americans suffer from a mental illness?
            How pharmacotherapy is used in the treatment of mental diseases?
            What are the early signs related to mental health difficulties?
            What are the two most common mental disorders?
            What is the estimated global cost of mental illness?
            Who developed the taxonomy of mental disorders?
            What is the conclusion of Richards, Campania, & Muse-Burke's research?
            What did Clifford Beers open for the first time in United States?    '''
q_data

"How World Health Organization(WHO) defines mental health? \n            Who was the first person to coin the term mental hygiene? \n            What percentage of Americans suffer from a mental illness? \n            How pharmacotherapy is used in the treatment of mental diseases? \n            What are the early signs related to mental health difficulties? \n            What are the two most common mental disorders? \n            What is the estimated global cost of mental illness?\n            Who developed the taxonomy of mental disorders?\n            What is the conclusion of Richards, Campania, & Muse-Burke's research?\n            What did Clifford Beers open for the first time in United States?    "

### Splitting data into chunks (sentences) and creating sentence embeddings using Spark NLP

In [ ]:
# Creating Spark NLP pipeline which creates sentences and sentence embeddings from raw data
def embeddings_pipeline():
  # Annotator that transforms a text column from dataframe into an Annotation ready for NLP
  documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

  sentenceEmbeddings = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_uncased","en")\
    .setInputCols(["sentence"])\
    .setOutputCol("sentence_embeddings")

  nlpPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    sentenceEmbeddings])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  return model
  # df = spark.createDataFrame([[data]]).toDF("text")

  # model = nlpPipeline.fit(df)
  # result_bert = model.transform(df)

  # result_df = result_bert.select(F.explode(F.arrays_zip
  #                         (result_bert.sentence.result,
  #                          result_bert.sentence_embeddings.embeddings)).alias("cols")) \
  #                       .select(F.expr("cols['0']").alias("sentence"),
  #                               F.expr("cols['1']").alias("sentence_embeddings"))
  # return result_df

In [ ]:
# creating dataframes for context and questions
def create_df(model,datatype):
  id = []
  sentences = []
  embeddings = []

  total_sentences = model[0]['sentence']
  total_embeddings = model[0]['sentence_embeddings']

  for i in range(len(total_sentences)):
    sentences.append(total_sentences[i].result)
    id.append(str(int(total_sentences[i].metadata['sentence'])+1))

  for i in range(len(total_embeddings)):
    embeddings.append(total_embeddings[i].embeddings)

  if datatype == 'context':
    df = pd.DataFrame({'id': id, "sentences": sentences, "sentence_embeddings": embeddings})
  elif datatype == 'question':
    df = pd.DataFrame({'id': id, "questions": sentences, "question_embeddings": embeddings})

  return df

In [ ]:
def generate_embeddings_df(model,data,q_data):

  # creating context and question models with LightPipeline
  light_model_with_emb = nlp.LightPipeline(model, parse_embeddings=True)

  context_model = light_model_with_emb.fullAnnotate(data)
  question_model = light_model_with_emb.fullAnnotate(q_data)

  # creating context and question dataframes
  context_df = create_df(context_model, "context")
  question_df = create_df(question_model, "question")

  return context_df, question_df

In [ ]:
# calling above functions to create context and question datasets with embeddings
model = embeddings_pipeline()
context_df, question_df = generate_embeddings_df(model,data,q_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
sent_bert_base_uncased download started this may take some time.
Approximate size to download 392.5 MB
[OK!]


### Save&Load document embeddings

In [ ]:
# save content and question dfs to drive
context_df.to_csv('/content/drive/MyDrive/RAG/context.csv', index=False)
question_df.to_csv('/content/drive/MyDrive/RAG/question.csv', index=False)

In [ ]:
# load content and question dfs from drive
context_df = pd.read_csv('/content/drive/MyDrive/RAG/context.csv')
question_df = pd.read_csv('/content/drive/MyDrive/RAG/question.csv')

In [ ]:
context_df.head()

,id,sentences,sentence_embeddings
0,1,"Mental health encompasses emotional, psycholog...","[0.2077588, 0.3205036, 0.15082607, -0.05214445..."
1,2,"According to World Health Organization (WHO), ...","[-0.10995298, 0.27855635, 0.24241367, -0.22153..."
2,3,It likewise determines how an individual handl...,"[0.04802175, 0.09302781, -0.10260814, -0.00403..."
3,4,"Mental health includes subjective well-being, ...","[-0.02883486, 0.26770076, 0.088557035, 0.02614..."
4,5,From the perspectives of positive psychology o...,"[0.19019742, 0.23576315, -0.07932275, -0.02739..."


In [ ]:
question_df.head()

,id,questions,question_embeddings
0,1,How World Health Organization(WHO) defines men...,"[0.05972476, 0.12781732, -0.25980595, -0.04823..."
1,2,Who was the first person to coin the term ment...,"[0.0069822036, 0.18677203, -0.39095375, -0.037..."
2,3,What percentage of Americans suffer from a men...,"[0.090737164, 0.15851025, -0.16805732, -0.0314..."
3,4,How pharmacotherapy is used in the treatment o...,"[-0.074681826, 0.29530752, -0.3378918, -0.0415..."
4,5,What are the early signs related to mental hea...,"[-0.034740552, 0.22984867, -0.005064979, -0.04..."


## Step 2: Augmentation

### Indexing documents by using Vector database ChromaDB and retrieve the most relevant context information for the given query

In [ ]:
# get the chroma client
import chromadb
chroma_client = chromadb.Client()

In [ ]:
# create a chromadb collection to store context embeddings
def create_collection(chroma_client, context_df):
  # create a collection
  collection = chroma_client.create_collection(name="mental_health")
  # add vector data to the collection
  collection.add(
      embeddings= context_df['sentence_embeddings'].apply(ast.literal_eval).tolist(),
      documents= context_df['sentences'].tolist(),
      #metadatas=[{"source": "my_source"}, {"source": "my_source"}],
      ids=context_df['id'].astype(str).tolist()  # Convert ID to string
  )
  return collection

In [ ]:
collection = create_collection(chroma_client, context_df)

In [ ]:
# combine relevant context information with the initial question prompt
def construct_augmented_prompt(question_df,collection):
  # list of prompts for each question/context pair
  context_prompt_list = []
  question_prompt_list = []
  # retrieving questions and their embeddings
  question_list = question_df['questions'].tolist()
  question_list_emb = question_df['question_embeddings'].apply(ast.literal_eval).tolist()
  # for each question, query chroma db and obtain 15 most appropriate sentences from the dataset
  for i in range(len(question_list)):
    # initialize prompts
    context_prompt = ""
    question_prompt = question_list[i]
    # find top 15 documents that are similar to query for the context
    results = collection.query(
        query_embeddings = question_list_emb[i],
        n_results = 15
    )
    relevant_context = results["documents"][0]
    for sent in relevant_context:
      context_prompt += sent + " "
    # store prompts in lists
    context_prompt_list.append(context_prompt)
    question_prompt_list.append(question_prompt)

  return context_prompt_list, question_prompt_list

In [ ]:
context_prompts, question_prompts = construct_augmented_prompt(question_df,collection)

In [ ]:
context_prompts[0]

'Investigating the degree and severity of untreated emotional mental disorders throughout the world is a top priority of the World Mental Health (WMH) survey initiative, which was created in 1998 by the World Health Organization (WHO). The World Mental Health survey initiative has suggested a plan for countries to redesign their mental health care systems to best allocate resources. Prevention is beginning to appear in mental health strategies, including the 2004 WHO report "Prevention of Mental Disorders", the 2008 EU "Pact for Mental Health" and the 2011 US National Prevention Strategy. Emotional mental illnesses should be a particular concern in the United States since the U.S. has the highest annual prevalence rates (26 percent) for mental illnesses among a comparison of 14 developing and developed countries. World Mental Health Day Mental health is a socially constructed concept; different societies, groups, cultures, institutions, and professions have very different ways of conce

In [ ]:
question_prompts[0]

'How World Health Organization(WHO) defines mental health?'

### Save&Load Extracted Relevant Context Prompts and Question prompts for Comparison Purposes

In [ ]:
import json
# save in a text file
with open('/content/drive/MyDrive/RAG/context_prompts.json','w') as file:
  json.dump(context_prompts,file)
with open('/content/drive/MyDrive/RAG/question_prompts.json','w') as file:
  json.dump(question_prompts,file)

In [ ]:
import json
#load it back to list
with open('/content/drive/MyDrive/RAG/context_prompts.json', 'r') as file:
    context_prompts = json.load(file)
with open('/content/drive/MyDrive/RAG/question_prompts.json', 'r') as file:
    question_prompts = json.load(file)

## Step 3: Generation

### Augmented prompt is fed to LLM to generate answer using Spark NLP Medical QA models

In [ ]:
def medical_qa_pipeline(model_name):
  # Annotator that transforms a text column from dataframe into an Annotation ready for NLP
  documentAssembler = nlp.MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

  medQA  = medical.MedicalQuestionAnswering().pretrained(model_name, "en", "clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Context: {context} \n Question: {question} \n Answer: ")\
    .setOutputCol("answer")\

  nlpPipeline = nlp.Pipeline(stages=[documentAssembler, medQA])

  empty_data = spark.createDataFrame([["",""]]).toDF("question","context")

  model = nlpPipeline.fit(empty_data)

  return model

In [ ]:
def generate_qa_df(model,question_prompts,context_prompts):
  light_model = nlp.LightPipeline(model)

  # initialize list of questions, context and answers
  question_lst = []
  context_lst = []
  answer_lst = []

  for i in range(len(context_prompts)):

    light_result = light_model.annotate([question_prompts[i]],[context_prompts[i]])
    cur_question = light_result[0]['document_question'][0]
    cur_context = light_result[0]['document_context'][0]
    cur_answer = light_result[0]['answer'][0]

    question_lst.append(cur_question)
    context_lst.append(cur_context)
    answer_lst.append(cur_answer)

  # creating dataframe
  qa_df = pd.DataFrame({'question': question_lst, "context": context_lst, "answer": answer_lst})
  return qa_df

### Flan T5

In [ ]:
t5_model = medical_qa_pipeline("flan_t5_base_jsl_qa")
# save the model
#t5_model.write().overwrite().save("/content/drive/MyDrive/RAG/flan_t5_model")

flan_t5_base_jsl_qa download started this may take some time.
[OK!]


In [ ]:
qa_t5_df = generate_qa_df(t5_model,question_prompts,context_prompts)
qa_t5_df.to_csv('/content/drive/MyDrive/RAG/qa_t5.csv', index=False)

In [ ]:
qa_t5_df.head()

,question,context,answer
0,How World Health Organization(WHO) defines men...,Investigating the degree and severity of untre...,The World Health Organization (WHO) defines me...
1,Who was the first person to coin the term ment...,"In the mid-19th century, William Sweetser was ...",William Sweetser
2,What percentage of Americans suffer from a men...,"In 2019, about 970 million people worldwide, s...",Over 26 percent of all Americans over the age ...
3,How pharmacotherapy is used in the treatment o...,Pharmacotherapy is used in the treatment of me...,Pharmacotherapy is used in the treatment of me...
4,What are the early signs related to mental hea...,There are many things that can contribute to m...,"Sleep irritation, lack of energy, lack of appe..."


### Clinical Notes QA

In [ ]:
clinical_model = medical_qa_pipeline("clinical_notes_qa_base")
# save the model
clinical_model.write().overwrite().save("/content/drive/MyDrive/RAG/clinical_qa_model")

clinical_notes_qa_base download started this may take some time.
[OK!]


In [ ]:
qa_clinical_df = generate_qa_df(clinical_model,question_prompts,context_prompts)
qa_clinical_df.to_csv('/content/drive/MyDrive/RAG/qa_clinical.csv', index=False)

In [ ]:
qa_clinical_df.head()

,question,context,answer
0,How World Health Organization(WHO) defines men...,Investigating the degree and severity of untre...,"Mental health is a ""state of well-being in whi..."
1,Who was the first person to coin the term ment...,"In the mid-19th century, William Sweetser was ...",William Sweetser was the first person to coin ...
2,What percentage of Americans suffer from a men...,"In 2019, about 970 million people worldwide, s...",Over 26 percent of all Americans over the age ...
3,How pharmacotherapy is used in the treatment o...,Pharmacotherapy is used in the treatment of me...,Pharmacotherapy is used in the treatment of me...
4,What are the early signs related to mental hea...,There are many things that can contribute to m...,"Sleep irritation, lack of energy, lack of appe..."


### Non-medical QA Models

In [ ]:
def qa_pipeline(model_name):
  # Annotator that transforms a text column from dataframe into an Annotation ready for NLP
  documentAssembler = nlp.MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

  if model_name == 'albert':
    QA  = nlp.AlbertForQuestionAnswering().pretrained()\
      .setInputCols(["document_question", "document_context"])\
      .setOutputCol("answer") \
      .setCaseSensitive(False)
  elif model_name == 'bert':
    QA  = nlp.BertForQuestionAnswering().pretrained()\
      .setInputCols(["document_question", "document_context"])\
      .setOutputCol("answer") \
      .setCaseSensitive(False)
  elif model_name == "roberta":
    QA  = nlp.RoBertaForQuestionAnswering().pretrained()\
      .setInputCols(["document_question", "document_context"])\
      .setOutputCol("answer") \
      .setCaseSensitive(False)

  nlpPipeline = nlp.Pipeline(stages=[documentAssembler, QA])

  empty_data = spark.createDataFrame([["",""]]).toDF("question","context")

  model = nlpPipeline.fit(empty_data)

  return model

## Albert

In [ ]:
albert_model = qa_pipeline("albert")

albert_base_qa_squad2 download started this may take some time.
Approximate size to download 40.5 MB
[OK!]


In [ ]:
qa_albert_df = generate_qa_df(albert_model,question_prompts,context_prompts)
qa_albert_df.to_csv('/content/drive/MyDrive/RAG/qa_albert.csv', index=False)

In [ ]:
qa_albert_df.head()

,question,context,answer
0,How World Health Organization(WHO) defines men...,Investigating the degree and severity of untre...,
1,Who was the first person to coin the term ment...,"In the mid-19th century, William Sweetser was ...",william sweetser
2,What percentage of Americans suffer from a men...,"In 2019, about 970 million people worldwide, s...",over 26 percent
3,How pharmacotherapy is used in the treatment o...,Pharmacotherapy is used in the treatment of me...,"through the use of antidepressants,benzodiazep..."
4,What are the early signs related to mental hea...,There are many things that can contribute to m...,"sleep irritation, lack of energy, lack of appe..."


## Bert

In [ ]:
bert_model = qa_pipeline("bert")

bert_base_cased_qa_squad2 download started this may take some time.
Approximate size to download 385.5 MB
[OK!]


In [ ]:
qa_bert_df = generate_qa_df(bert_model,question_prompts,context_prompts)
qa_bert_df.to_csv('/content/drive/MyDrive/RAG/qa_bert.csv', index=False)

In [ ]:
qa_bert_df.head()

,question,context,answer
0,How World Health Organization(WHO) defines men...,Investigating the degree and severity of untre...,WHO
1,Who was the first person to coin the term ment...,"In the mid-19th century, William Sweetser was ...",William Sweetser
2,What percentage of Americans suffer from a men...,"In 2019, about 970 million people worldwide, s...",Over 26
3,How pharmacotherapy is used in the treatment o...,Pharmacotherapy is used in the treatment of me...,"through the use of antidepressants , benzodiaz..."
4,What are the early signs related to mental hea...,There are many things that can contribute to m...,"sleep irritation , lack of energy , lack of ap..."


## Roberta

In [ ]:
roberta_model = qa_pipeline("roberta")

roberta_base_qa_squad2 download started this may take some time.
Approximate size to download 442.6 MB
[OK!]


In [ ]:
qa_roberta_df = generate_qa_df(roberta_model,question_prompts,context_prompts)
qa_roberta_df.to_csv('/content/drive/MyDrive/RAG/qa_roberta.csv', index=False)

In [ ]:
qa_roberta_df.head()

,question,context,answer
0,How World Health Organization(WHO) defines men...,Investigating the degree and severity of untre...,
1,Who was the first person to coin the term ment...,"In the mid-19th century, William Sweetser was ...",william sweetser
2,What percentage of Americans suffer from a men...,"In 2019, about 970 million people worldwide, s...",over 26 percent
3,How pharmacotherapy is used in the treatment o...,Pharmacotherapy is used in the treatment of me...,"through the use of antidepressants , benzodiaz..."
4,What are the early signs related to mental hea...,There are many things that can contribute to m...,"sleep irritation , lack of energy , lack of ap..."


### Combine results of 2 medical and 3 non-medical qa models in a single dataframe to compare answers

In [ ]:
def combine_dfs(df1,df2,df3,df4,df5):
  # First, rename the answer columns in each dataframe to be unique
  df1 = df1.rename(columns={'answer': 'flan_t5_answer'})
  df2 = df2.rename(columns={'answer': 'clinical_qa_answer'})
  df3 = df3.rename(columns={'answer': 'albert_answer'})
  df4 = df4.rename(columns={'answer': 'bert_answer'})
  df5 = df5.rename(columns={'answer': 'roberta_answer'})

  # Now, merge the dataframes on the question and context columns
  merged_df = df1.merge(df2, on=['question', 'context'])
  merged_df = merged_df.merge(df3, on=['question', 'context'])
  merged_df = merged_df.merge(df4, on=['question', 'context'])
  merged_df = merged_df.merge(df5, on=['question', 'context'])

  # save the combined df to csv
  merged_df.to_csv('/content/drive/MyDrive/RAG/qa_combined.csv', index=False)
  return merged_df

In [ ]:
# load content and question dfs from drive
qa_t5_df = pd.read_csv('/content/drive/MyDrive/RAG/qa_t5.csv')
qa_clinical_df = pd.read_csv('/content/drive/MyDrive/RAG/qa_clinical.csv')
qa_albert_df = pd.read_csv('/content/drive/MyDrive/RAG/qa_albert.csv')
qa_bert_df = pd.read_csv('/content/drive/MyDrive/RAG/qa_bert.csv')
qa_roberta_df = pd.read_csv('/content/drive/MyDrive/RAG/qa_roberta.csv')

In [ ]:
merged_df = combine_dfs(qa_t5_df,qa_clinical_df,qa_albert_df,qa_bert_df,qa_roberta_df)
merged_df.head()

,question,context,flan_t5_answer,clinical_qa_answer,albert_answer,bert_answer,roberta_answer
0,How World Health Organization(WHO) defines men...,Investigating the degree and severity of untre...,The World Health Organization (WHO) defines me...,"Mental health is a ""state of well-being in whi...",NaN,WHO,NaN
1,Who was the first person to coin the term ment...,"In the mid-19th century, William Sweetser was ...",William Sweetser,William Sweetser was the first person to coin ...,william sweetser,William Sweetser,william sweetser
2,What percentage of Americans suffer from a men...,"In 2019, about 970 million people worldwide, s...",Over 26 percent of all Americans over the age ...,Over 26 percent of all Americans over the age ...,over 26 percent,Over 26,over 26 percent
3,How pharmacotherapy is used in the treatment o...,Pharmacotherapy is used in the treatment of me...,Pharmacotherapy is used in the treatment of me...,Pharmacotherapy is used in the treatment of me...,"through the use of antidepressants,benzodiazep...","through the use of antidepressants , benzodiaz...","through the use of antidepressants , benzodiaz..."
4,What are the early signs related to mental hea...,There are many things that can contribute to m...,"Sleep irritation, lack of energy, lack of appe...","Sleep irritation, lack of energy, lack of appe...","sleep irritation, lack of energy, lack of appe...","sleep irritation , lack of energy , lack of ap...","sleep irritation , lack of energy , lack of ap..."
